In [11]:
import pandas as pd
import os
from datetime import datetime

In [12]:
R_Ms_ADRES_EPS025 = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\04_Abril\15\EPS025MS0014042025.TXT"
R_Ms_SIE = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\SIE\Aseguramiento\ms_sie\Reporte_Validación Archivos Maestro_2025_04_15.csv"
Salida = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\04_Abril\15\NS 15-04-2025.xlsx"
R_dicionario = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\04_Abril\15\NOMENCLATURA_DE_DIRECCIONES.xlsx"

In [13]:
Df_SIE = pd.read_csv(R_Ms_SIE, sep=';', dtype=str, encoding='ANSI')

new_columns = ["AFL_ID", "ENT_ID", "TPS_IDN_ID_CF", "HST_IDN_NUMERO_IDENTIFICACION_CF", "TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION", "AFL_PRIMER_APELLIDO", "AFL_SEGUNDO_APELLIDO", "AFL_PRIMER_NOMBRE", "AFL_SEGUNDO_NOMBRE", "AFL_FECHA_NACIMIENTO", "TPS_GNR_ID", "AFL_PAIS_NACIMIENTO", "AFL_MUNICIPIO_NACIMIENTO", "AFL_NACIONALIDAD", "AFL_SEXO_IDENTIFICACION", "AFL_DISCAPACIDAD", "TPS_AFL_ID", "TPS_PRN_ID", "TPS_GRP_PBL_ID", "TPS_NVL_SSB_ID", "NUMEROFICHASISBEN", "TPS_CND_BNF_ID", "DPR_ID", "MNC_ID", "ZNS_ID", "AFL_FECHA_AFILIACION_SGSSS", "AFC_FECHA_INICIO", "NUMERO CONTRATO", "FECHADE INICIO DEL CONTRATO", "CNT_AFL_TPS_GRP_PBL_ID", "CNT_AFL_TPS_PRT_ETN_ID", "TPS_MDL_SBS_ID", "TPS_EST_AFL_ID", "CND_AFL_FECHA_INICIO", "CND_AFL_FECHA_INICIO", "GRP_FML_COTIZANTE_ID", "PORTABILIDAD", "COD_IPS_P", "MTDLG_G_P", "SUB_SISBEN_IV", "MARCASISBENIV+MARCASISBENIII", "CRUCE_BDEX_RNEC"]

Df_EPS025 = pd.read_csv(R_Ms_ADRES_EPS025, sep=',', header=None, dtype=str, encoding='ANSI')
Df_EPS025.columns = new_columns

In [14]:
df_nomenclatura = pd.read_excel(R_dicionario, sheet_name="NOMENCLATURA", header=0, dtype=str)
df_nomenclatura.columns

Index(['Nomclatura', 'Nombre'], dtype='object')

In [15]:
# Seleccionar las columnas clave y las columnas a agregar de Df_SIE
cols_to_transfer = ["tipo_documento", "numero_identificacion", "direccion", "celular", "correo_electronico", "discapacidad", "municipio_nacimiento"]
df_sie_subset = Df_SIE[cols_to_transfer]

# Realizar la fusión (merge) de Df_EPS025 con el subconjunto de Df_SIE utilizando las columnas de identificación
Df_EPS025 = Df_EPS025.merge(
	df_sie_subset,
	how="left",
	left_on=["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"],
	right_on=["tipo_documento", "numero_identificacion"]
)

# Eliminar las columnas clave de Df_SIE que ya no sean necesarias
Df_EPS025.drop(columns=["tipo_documento", "numero_identificacion"], inplace=True)

In [16]:
Df_EPS025 = Df_EPS025[Df_EPS025["TPS_EST_AFL_ID"] == "AC"]

In [17]:
import difflib
import pandas as pd
import numpy as np

# Crear diccionario desde df_nomenclatura
nomenclatura_dict = dict(
    zip(
        df_nomenclatura["Nombre"].str.lower().str.strip(),
        df_nomenclatura["Nomclatura"].str.strip()
    )
)

def sustituir_palabras(direccion):
    # Verifica que el valor sea una cadena de texto
    if not isinstance(direccion, str):
        return np.nan  # Devuelve NaN para mantener consistencia en tipos

    palabras = direccion.split()
    resultado = []
    for palabra in palabras:
        coincidencias = difflib.get_close_matches(palabra.lower(), nomenclatura_dict.keys(), n=1, cutoff=0.8)
        if coincidencias:
            resultado.append(nomenclatura_dict[coincidencias[0]])
        else:
            resultado.append(palabra)
    return " ".join(resultado)

# Aplicar la transformación a la columna "direccion" de Df_EPS025
Df_EPS025["direccion"] = Df_EPS025["direccion"].apply(sustituir_palabras)


In [18]:
import re
import difflib

# Lista de palabras prohibidas (en minúsculas y sin espacios)
forbidden = ["notiene", "actualizar", "sincorreo"]

def limpiar_valor(val):
    if not isinstance(val, str):
        return val
    # Dividir el texto en tokens usando caracteres no alfanuméricos
    tokens = re.split(r'\W+', val.lower())
    for token in tokens:
        if token:  # Si el token no está vacío
            for palabra in forbidden:
                # Si la semejanza es mayor o igual al 80%, se limpia el dato
                if difflib.SequenceMatcher(None, token, palabra).ratio() >= 0.8:
                    return ""
    return val

# Aplicar la función de limpieza a las columnas "direccion", "celular" y "correo_electronico"
for col in ["direccion", "celular", "correo_electronico"]:
    Df_EPS025[col] = Df_EPS025[col].apply(limpiar_valor)

In [19]:
import re

def validate_celular(cell):
    if not isinstance(cell, str):
        return ""
    # Eliminar espacios y todos los caracteres que no sean dígitos
    cleaned = re.sub(r"[^0-9]", "", cell)
    # Si el resultado no tiene 10 dígitos o no inicia con 3, se asigna cadena vacía
    if len(cleaned) != 10 or not cleaned.startswith("3"):
        return ""
    return cleaned

Df_EPS025["celular"] = Df_EPS025["celular"].apply(validate_celular)

In [20]:
mask = Df_EPS025[["direccion", "celular", "correo_electronico", "discapacidad", "municipio_nacimiento"]].replace("", pd.NA).isna().all(axis=1)
Df_EPS025 = Df_EPS025[~mask]

In [21]:
Df_EPS025.to_excel(Salida, index=False)